# An analysis of the physicochemical properties of oral drugs from 2000 to 2022 

Daniel Cole, Rachael Pirie, Harriet A. Stanway-Gordon, Hannah L. Stewart, Kirsty L. Wilson and Michael J. Waring

In [1]:
# general imports

import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Descriptors

from helpers import *
from confgen import GenConformerEnsemble

In [2]:
# load data and set up dataframe
data = pd.read_excel("../data/FDA_0022.xlsx")
data['ROMol'] = data['SMILES'].apply(Chem.MolFromSmiles)  # get 2D

### Lipinski Descriptors

In [3]:
from rdkit.Chem import Descriptors, Lipinski

# get original Ro5 properties
data['MW'] = data['ROMol'].apply(Descriptors.ExactMolWt)  # molecular weight
data['no. HBAs'] = data['ROMol'].apply(Descriptors.NumHAcceptors)  # number of hydrogen bond acceptors
data['no. HBDs'] = data['ROMol'].apply(Descriptors.NumHDonors)  # number of hydrogen bond donors
data['no. NO'] = data['ROMol'].apply(Lipinski.NOCount)  # correct HBA count for Lipinski's rules
data['no. NHOH'] = data['ROMol'].apply(Lipinski.NHOHCount)  # correct HBD count for Lipinski's rules
data['calc LogP'] = data['ROMol'].apply(Descriptors.MolLogP)  # calculated LogP, add (mol, True) to include Hs

### Atom/Bond Count Descriptors

In [4]:
from rdkit.Chem import rdMolDescriptors

# get atom/bond descriptors
data['no. RBs'] = data['ROMol'].apply(rdMolDescriptors.CalcNumRotatableBonds)  # number of rotable bonds
data['no. Aromatic Rings'] = data['ROMol'].apply(rdMolDescriptors.CalcNumAromaticRings)  # number of aromatic rings
data['no. stereocentres'] = data['ROMol'].apply(rdMolDescriptors.CalcNumAtomStereoCenters)  # number of stereocentres
data['no. small rings'] = data['ROMol'].apply(rdMolDescriptors.CalcNumRings)  # number of small rings
data['is macrocycle'] = data['ROMol'].apply(GetIsMacrocycle)  # check for macrocycles

### Atom Count Descriptors

In [5]:
from rdkit.Chem import HybridizationType
from rdkit.Chem.Fragments import fr_amide, fr_Ar_N, fr_Ar_NH, fr_NH0, fr_NH1, fr_NH2

# get number of atom-type descriptors
data['no. atoms'] = [mol.GetNumAtoms() for mol in list(data['ROMol'])]  # number of non-H atoms
data['no. non-C atoms'] = [len([(atom.GetSymbol() != 'C') for atom in mol.GetAtoms()]) for mol in list(data['ROMol'])]  # number of non C/H atoms
data['no. aromatic atoms'] = [sum([atom.GetIsAromatic() for atom in mol.GetAtoms()]) for mol in list(data['ROMol'])]  # number of aromatic atoms
data['no. sp3 atoms'] = [sum((atom.GetHybridization() == HybridizationType.SP3) for atom in mol.GetAtoms()) for mol in list(data['ROMol'])]  # number of sp3 atoms
data['no. amides'] = [fr_amide(mol) for mol in list(data['ROMol'])]  # amide count
data['no. Aromatic Nitrogens'] = [fr_Ar_N(mol) for mol in list(data['ROMol'])]  # aromatic N count
data['no. Aromatic Amines'] = [fr_Ar_NH(mol) for mol in list(data['ROMol'])]  # aromatic amine count
data['no. amines'] = [fr_NH0(mol) + fr_NH1(mol) + fr_NH2(mol) for mol in list(data['ROMol'])]  # amine count
data['no. electronegative atoms'] = data['ROMol'].apply(CalcNumElectronegative)

### Surface Area Descriptors

In [6]:
from rdkit.Chem import MolSurf

# get Surface area descriptors
data['approx. SA'] = [MolSurf.pyLabuteASA(mol, includeHs=1) for mol in list(data['ROMol'])]  # approximation to total surface area
data['PSA'] = [MolSurf.TPSA(mol) for mol in list(data['ROMol'])]  
data['relative SA'] = data['PSA'] / data['approx. SA']

### Other

In [7]:
from rdkit.Chem import QED, GetDistanceMatrix
from rdkit.Chem.GraphDescriptors import BertzCT

# get other descriptors
data['QED Druglikeness'] = [QED.default(mol) for mol in list(data['ROMol'])]  # this is a drug-likeness score,0 (not) and 1 (very)
data['molecular complexity'] = [BertzCT(mol, cutoff=100, dMat=None, forceDMat=1) for mol in list(data['ROMol'])]  # molecular complexity
data['Shape Index'] = (data['ROMol'].apply(GetDistanceMatrix).apply(np.max)) / data['no. atoms'] 

In [8]:
# add index as MolNo for merge with 3D props
data['MolNo'] = data.index.values

### 3D Shape Descriptors

In [9]:
from zipfile import ZipFile
from rdkit.Chem import Descriptors3D, PandasTools

In [ ]:
# generate ensemble if needed
GenConformerEnsemble("../data/FDA_0022.xlsx", "../data/FDA_0022_confs.sdf")

In [10]:
# load ensemble
# compressed using zip to meet GitHub upload size
#with ZipFile("../data/FDA_0022_confs.sdf.zip", 'r') as zip_ref:
    #zip_ref.extractall("../data/")

# load using PandasTools
data_confs = PandasTools.LoadSDF("../data/FDA_0022_confs.sdf", idName=None)

# convert mol id and no. rotatable bonds to int
data_confs['MolNo'] = data_confs['MolNo'].astype(int)  
data_confs['No RBs'] = data_confs['No RBs'].astype(int)

In [11]:
# retrieve cases where no 3D confs generated and split out
# usually these have either boron or metal, RDKit doesn't handle well
no_3D_multi = data_confs[data_confs['Energy'] == '-']
no_3D_multi = no_3D_multi.drop_duplicates(subset='MolNo', ignore_index=True)

# get properties from original dataframe and merge
no_3D_multi_props = data[data.index.isin(list(no_3D_multi['MolNo']))]
no_3D_multi = no_3D_multi_props.merge(no_3D_multi, on='MolNo')
no_3D_multi = no_3D_multi.drop(columns=['CID', 'Energy', 'ROMol_x', 'ROMol_y'])  # remove cols not present in final

# remove failed conf gen from data
data_confs = data_confs[data_confs['Energy'] != '-']
data_confs['Energy'] = data_confs['Energy'].astype(float)
data = data[~data.index.isin(list(no_3D_multi['MolNo']))]

In [12]:
# convert to same shape as data
grouped_df = pd.DataFrame()

grouped_df['MolNo'] = list(set(list(data_confs['MolNo'])))
grouped_df['No Confs'] = [data_confs.loc[data_confs['MolNo'] == i, 'No Confs'].iloc[0] for i in list(grouped_df['MolNo'])]
grouped_df['No RBs'] = [data_confs.loc[data_confs['MolNo'] == i, 'No RBs'].iloc[0] for i in list(grouped_df['MolNo'])]
grouped_df['CID'] = [list(data_confs.loc[data_confs['MolNo'] == i, 'CID']) for i in list(grouped_df['MolNo'])]
grouped_df['Energy'] = [list(data_confs.loc[data_confs['MolNo'] == i, 'Energy']) for i in list(grouped_df['MolNo'])]
grouped_df['ROMol'] = [list(data_confs.loc[data_confs['MolNo'] == i, 'ROMol']) for i in list(grouped_df['MolNo'])]

In [13]:
grouped_df.shape

(381, 6)

In [14]:
no_3D_multi.shape

(1, 45)

In [15]:
data['No Confs'] = list(grouped_df['No Confs'])
data['No RBs'] = list(grouped_df['No RBs'])

# Radius of Gyration
rog = [[Descriptors3D.RadiusOfGyration(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]  # RoG
data['avg. RoG'] = [CalcBoltzAvg(row.Energy, rog[i]) for i, row in grouped_df.iterrows()]
data['min. RoG'] = [min(r) for r in rog]
data['max. RoG'] = [max(r) for r in rog]

# Asphericity
aspher = [[Descriptors3D.Asphericity(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]
data['avg. asphericity'] = [CalcBoltzAvg(row.Energy, aspher[i]) for i, row in grouped_df.iterrows()]
data['min. asphericity'] = [min(r) for r in aspher]
data['max. asphericity'] = [max(r) for r in aspher]

# Eccentricity
eccen = [[Descriptors3D.Eccentricity(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]
data['avg. eccentricity'] = [CalcBoltzAvg(row.Energy, eccen[i]) for i, row in grouped_df.iterrows()]
data['min. eccentricity'] = [min(r) for r in eccen]
data['max. eccentricity'] = [max(r) for r in eccen]

# Inertial Shape Factor
inertial = [[Descriptors3D.InertialShapeFactor(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]
data['avg. inertial shape factor'] = [CalcBoltzAvg(row.Energy, inertial[i]) for i, row in grouped_df.iterrows()]
data['min. inertial shape factor'] = [min(r) for r in inertial]
data['max. inertial shape factor'] = [max(r) for r in inertial]

# Molecular Spherocity Index
spherocity = [[Descriptors3D.SpherocityIndex(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]
data['avg. Molecular Spherocity Index'] = [CalcBoltzAvg(row.Energy, spherocity[i]) for i, row in grouped_df.iterrows()]
data['min. Molecular Spherocity Index'] = [min(r) for r in spherocity]
data['max. Molecular Spherocity Index'] = [max(r) for r in spherocity]

# Normalized principal moments ratio 1
npr1 = [[Descriptors3D.NPR1(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]
data['avg. PMI1/PMI3'] = [CalcBoltzAvg(row.Energy, npr1[i]) for i, row in grouped_df.iterrows()]
data['min. PMI1/PMI3'] = [min(r) for r in npr1]
data['max. PMI1/PMI3'] = [max(r) for r in npr1]

# Normalized principal moments ratio 2
npr2 = [[Descriptors3D.NPR2(mol) for mol in mols] for mols in list(grouped_df['ROMol'])]
data['avg. PMI2/PMI3'] = [CalcBoltzAvg(row.Energy, npr2[i]) for i, row in grouped_df.iterrows()]
data['min. PMI2/PMI3'] = [min(r) for r in npr2]
data['max. PMI2/PMI3'] = [max(r) for r in npr2]

In [16]:
# drop mol object column for outputs
data = data.drop(columns=['ROMol'])

In [17]:
# set placeholders for no 3D conformers
no_3D_multi['No Confs'] = '-'
no_3D_multi['No RBs'] = '-'
no_3D_multi['avg. RoG'] = '-'
no_3D_multi['min. RoG'] = '-'
no_3D_multi['max. RoG'] = '-'
no_3D_multi['avg. asphericity'] = '-'
no_3D_multi['min. asphericity'] = '-'
no_3D_multi['max. asphericity'] = '-'
no_3D_multi['avg. eccentricity'] = '-'
no_3D_multi['min. eccentricity'] = '-'
no_3D_multi['max. eccentricity'] = '-'
no_3D_multi['avg. inertial shape factor'] = '-'
no_3D_multi['min. inertial shape factor'] = '-'
no_3D_multi['max. inertial shape factor'] = '-'
no_3D_multi['avg. Molecular Spherocity Index'] = '-'
no_3D_multi['min. Molecular Spherocity Index'] = '-'
no_3D_multi['max. Molecular Spherocity Index'] = '-'
no_3D_multi['avg. PMI1/PMI3'] = '-'
no_3D_multi['min. PMI1/PMI3'] = '-'
no_3D_multi['max. PMI1/PMI3'] = '-'
no_3D_multi['avg. PMI2/PMI3'] = '-'
no_3D_multi['min. PMI2/PMI3'] = '-'
no_3D_multi['max. PMI2/PMI3'] = '-'

In [24]:
# collate results
frames = [data, no_3D_multi]
data = pd.concat(frames, ignore_index=True)

# save to excel
data.to_excel("../data/props_avg_0022.xlsx", index=False)

In [25]:
data

,SMILES,Drug Name,Active ingredient,Date (D/M/Y),Year,Indication,Summary Indication,Protein target,DOI,Class,...,min. Molecular Spherocity Index,max. Molecular Spherocity Index,avg. PMI1/PMI3,min. PMI1/PMI3,max. PMI1/PMI3,avg. PMI2/PMI3,min. PMI2/PMI3,max. PMI2/PMI3,None,ID
0,COC1=C(C(=NC=C1)CS(=O)C2=NC3=C(N2)C=C(C=C3)OC(...,PROTONIX,PANTOPRAZOLE,2000-02-02 00:00:00,2000,"The treatment of stomach ulcers, short-term tr...",NaN,"(H+ , K+ )-ATPase",https://pubchem.ncbi.nlm.nih.gov/compound/Pant...,Proton-pump inhibitors,...,0.037227,0.387609,0.211142,0.074357,0.405906,0.854018,0.795833,0.983603,NaN,NaN
1,C1=CC(=CC=C1C(=O)NCCC(=O)O)N=NC2=CC(=C(C=C2)O)...,COLAZAL,BALSALAZIDE,18/07/2000,2000,The treatment of mildly to moderately active u...,Inflammatory bowel disease,Aminosalicylates,https://pubchem.ncbi.nlm.nih.gov/compound/Bals...,NaN,...,0.022991,0.331035,0.077548,0.045962,0.540218,0.969977,0.692322,0.998872,NaN,NaN
2,O=C([C@@H]1CC[C@@H](C(C)C)CC1)N[C@@H](C(O)=O)C...,STARLIX,NATEGLINIDE,22/12/2000,2000,To improve glycemic control in adults with typ...,Type 2 diabetes,ATP-dependent potassium channels,https://pubchem.ncbi.nlm.nih.gov/compound/Nate...,Meglitinides,...,0.03717,0.359775,0.256791,0.107311,0.465231,0.887354,0.727165,0.964561,NaN,NaN
3,CC([C@@H](C(N[C@@H](CC1=CC=CC=C1)C[C@@H]([C@@H...,KALETRA,LOPINAVIR,15/09/2000,2000,The treatment of HIV-1 infection in adults and...,HIV-1,HIV-1 protease,https://pubchem.ncbi.nlm.nih.gov/compound/Lopi...,Protease inhibitors,...,0.113767,0.830511,0.493218,0.210252,0.842864,0.810014,0.590828,0.992873,NaN,NaN
4,O=C(O[C@@H]1CNC(C)=O)N(C1)C2=CC(F)=C(N3CCOCC3)...,ZYVOX,LINEZOLID,18/04/2000,2000,The treatment of infections caused by Gram-pos...,Gram-positive bacteria,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/Line...,Oxazolidinones,...,0.018484,0.215406,0.192498,0.081461,0.223436,0.927574,0.863981,0.991655,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,CN1CCCC1COC2=NC3=C(CCN(C3)C4=CC=CC5=C4C(=CC=C5...,Krazati,Adagrasib,2022-12-12 00:00:00,2022,To treat KRAS G12C-mutated locally advanced or...,Lung Cancer,KRAS,10.1093/ajhp/zxad016,GTPase inhibitor,...,0.063667,0.434959,0.390085,0.244098,0.544896,0.754073,0.608791,0.957645,NaN,NaN
380,CC(C)(C#CC1=NC(=C(C=C1)C2=C3C(=C(C=C2)Cl)C(=NN...,Sunlenca,Lenacapavir,2022-12-22 00:00:00,2022,To treat adults with HIV whose HIV infections ...,HIV,p24 antigen,10.1093/ajhp/zxad018,Capsid inhibitor,...,0.126761,0.733962,0.520318,0.283725,0.690841,0.848873,0.604738,0.973384,NaN,NaN
381,CC(C)C[C@H](NC(CNC(C1=C(Cl)C=CC(Cl)=C1)=O)=O)B...,NINLARO,IXAZOMIB,20/11/2015,2015,To treat people with multiple myeloma who have...,Multiple myeloma,Proteasome subunit beta type-5,https://pubchem.ncbi.nlm.nih.gov/compound/Ixaz...,Proteasome inhibitors,...,-,-,-,-,-,-,-,-,,NaN
382,CC(C)C[C@H](NC(CNC(C1=C(Cl)C=CC(Cl)=C1)=O)=O)B...,NINLARO,IXAZOMIB,20/11/2015,2015,To treat people with multiple myeloma who have...,Multiple myeloma,Proteasome subunit beta type-5,https://pubchem.ncbi.nlm.nih.gov/compound/Ixaz...,Proteasome inhibitors,...,-,-,-,-,-,-,-,-,,
